In [1]:
from hydradx.model.amm.omnipool_amm import OmnipoolState
from hydradx.model.amm.agents import Agent
from hydradx.model.processing import get_kraken_orderbook, get_omnipool_data, get_kraken_orderbooks_from_file
from hydradx.model.processing import get_omnipool_data_from_file
from hydradx.model.amm.centralized_market import CentralizedMarket
import hydradx.model.amm.arbitrage_agent as arb

from pprint import pprint

import json


Set `get_live_markets = True` to get live markets from Kraken. Set to `False` to use order books from file.

In [2]:
get_live_markets = False

In [3]:
# 2: Acala DAI
# 3: Acala WBTC
# 4: Acala WETH
# 5: DOT
# 9: ASTR
# 10: native USDT
# 11: iBTC
# 12: ZTG
# 13: CFG
# 14: BNC
# 15: vDOT
# 16: GLMR
# 17: INTR
# 18: Wormhole DAI
# 19: Wormhole WBTC
# 20: Wormhole WETH
# 100: 4-Pool

In [4]:
arb_file = "arbconfig.txt"
# arb_file = "arbconfig2.txt"
with open('config/' + arb_file, 'r') as json_file:
    arb_list = json.load(json_file)
    
for d in arb_list:
    d['tkns'] = tuple(d['tkns'])
    d['tkn_ids'] = tuple(d['tkn_ids'])
    d['order_book'] = tuple(d['order_book'])

ob_objs = {}
order_book_asset_list = []
buffer = {}

if get_live_markets:
    for arb_cfg in arb_list:
        tkn_pair = arb_cfg['order_book']
        buffer[(arb_cfg['tkn_ids'],tkn_pair)] = arb_cfg['buffer']
        if tkn_pair not in ob_objs:
            order_book_url = 'https://api.kraken.com/0/public/Depth?pair=' + tkn_pair[0] + tkn_pair[1]
            ob_objs[tkn_pair] = get_kraken_orderbook(tkn_pair, order_book_url, archive=True)
            for tkn in tkn_pair:
                if tkn not in order_book_asset_list:
                    order_book_asset_list.append(tkn)
                    
else:
    ob_objs = get_kraken_orderbooks_from_file("data/input/")
    for arb_cfg in arb_list:
        tkn_pair = arb_cfg['order_book']
        buffer[(arb_cfg['tkn_ids'],tkn_pair)] = arb_cfg['buffer']
        for tkn in tkn_pair:
            if tkn not in order_book_asset_list:
                order_book_asset_list.append(tkn)    
                


In [5]:
RPC="wss://hydradx-rpc.dwellir.com"
N = 24
            
if get_live_markets:
    asset_list, asset_map, tokens, fees = get_omnipool_data(RPC, N, archive=True)
else:
    asset_list, asset_map, tokens, fees = get_omnipool_data_from_file("data/input/")
    
buffer2 = {((asset_map[k[0][0]], asset_map[k[0][1]]),k[1]): v for k, v in buffer.items()}
pprint(tokens)



{'ASTR': {'LRNA': 10052.666399219233, 'liquidity': 4209558.806607721},
 'BNC': {'LRNA': 6611.170221040792, 'liquidity': 728813.1700280587},
 'CFG': {'LRNA': 9583.605283154326, 'liquidity': 581942.6487409393},
 'DAI': {'LRNA': 12729.170373814593, 'liquidity': 342704.3489093943},
 'DAI001': {'LRNA': 1407.281258338137, 'liquidity': 37585.53464434876},
 'DOT': {'LRNA': 74148.64528120212, 'liquidity': 409408.5096801377},
 'GLMR': {'LRNA': 5519.295202377258, 'liquidity': 644608.4426455151},
 'HDX': {'LRNA': 20832.30379193331, 'liquidity': 110230487.17111325},
 'INTR': {'LRNA': 8466.186563507488, 'liquidity': 13502523.860204168},
 'USDT': {'LRNA': 83846.63411263155, 'liquidity': 2270681.588047},
 'WBTC': {'LRNA': 50.601798806111, 'liquidity': 0.04374934},
 'WBTC001': {'LRNA': 41157.56470534763, 'liquidity': 31.57785299},
 'WETH': {'LRNA': 21570.158913662108, 'liquidity': 307.1702895389879},
 'WETH001': {'LRNA': 71485.9049494652, 'liquidity': 1020.4656799657863},
 'ZTG': {'LRNA': 4486.58039644

In [6]:


cex_fee = 0.0016
# buffer = 0.0010

lrna_fee={asset: fees[asset]['protocol_fee'] for asset in asset_list}
asset_fee={asset: fees[asset]['asset_fee'] for asset in asset_list}

# lrna_fee = 0.0
# asset_fee = 0.0
# cex_fee = 0.0
# buffer = 0.0


cex = CentralizedMarket(
    order_book=ob_objs,
    asset_list=order_book_asset_list,
    trade_fee=cex_fee
)

op_state = OmnipoolState(
    tokens=tokens,
    lrna_fee = lrna_fee,
    asset_fee = asset_fee,
    preferred_stablecoin='USDT',
)

init_state = op_state.copy()


In [8]:
cex1 = cex.copy()
op_state1 = op_state.copy()
all_swaps1 = arb.get_arb_swaps_simple(op_state1, cex1, buffer=buffer2)

pprint(all_swaps1)
pprint(len(all_swaps1))

cex2 = cex.copy()
op_state2 = op_state.copy()
all_swaps2 = arb.get_arb_swaps(op_state2, cex2, buffer=buffer2)
print("strategy 2")
pprint(all_swaps2)
pprint(len(all_swaps2))

[{'dex': {'trade': 'buy', 'buy_asset': 'DOT', 'sell_asset': 'DAI', 'price': 4.9291, 'amount': 155.91628134031868, 'max_sell': 764.9135395980477}, 'cex': {'trade': 'sell', 'buy_asset': 'USD', 'sell_asset': 'DOT', 'price': 4.92663545, 'amount': 155.91628134031868}}, {'dex': {'trade': 'buy', 'buy_asset': 'DOT', 'sell_asset': 'DAI', 'price': 4.9291, 'amount': 0.0002696942157345657, 'max_sell': 0.001326508641242981}, 'cex': {'trade': 'sell', 'buy_asset': 'USD', 'sell_asset': 'DOT', 'price': 4.92663545, 'amount': 0.0002696942157345657}}, {'dex': {'trade': 'buy', 'buy_asset': 'DOT', 'sell_asset': 'DAI001', 'price': 4.9291, 'amount': 46.95744896546559, 'max_sell': 229.52820872890948}, 'cex': {'trade': 'sell', 'buy_asset': 'USD', 'sell_asset': 'DOT', 'price': 4.92663545, 'amount': 46.95744896546559}}, {'dex': {'trade': 'buy', 'buy_asset': 'DOT', 'sell_asset': 'WETH', 'price': 0.0026, 'amount': 67.20106071261026, 'max_sell': 0.1742282391786575}, 'cex': {'trade': 'sell', 'buy_asset': 'ETH', 'sell

In [9]:
state1 = init_state.copy()
state2 = init_state.copy()
holdings = {asset: 1000000000 for asset in asset_list + order_book_asset_list}
init_agent = Agent(holdings=holdings, unique_id='bot')
agent1 = init_agent.copy()
agent2 = init_agent.copy()
cex1 = cex.copy()
cex2 = cex.copy()

arb.execute_arb(state1, cex1, agent1, all_swaps1)
arb.execute_arb(state2, cex2, agent2, all_swaps2)

asset_map = {'WETH': 'ETH',
             'XETH': 'ETH',
             'XXBT': 'BTC',
             'WBTC': 'BTC',
             'ZUSD': 'USD',
             'USDT': 'USD',
             'USDC': 'USD',
             'DAI': 'USD',
             'USDT001': 'USD',
             'DAI001': 'USD',
             'WETH001': 'ETH',
             'WBTC001': 'BTC',
             'iBTC': 'BTC',
             'XBT': 'BTC',}

profit1 = arb.calculate_profit(init_agent, agent1, asset_map)
profit2 = arb.calculate_profit(init_agent, agent2, asset_map)

asset_map_omnipool = {
    'ETH': 'WETH',
    'XETH': 'WETH',
    'XXBT': 'WBTC',
    'BTC': 'WBTC',
    'ZUSD': 'USDT',
    'USD': 'USDT',
    'USDC': 'USDT',
    'DAI': 'USDT',
    'USDT001': 'USDT',
    'DAI001': 'USDT',
    'WETH001': 'WETH',
    'WBTC001': 'WBTC',
    'iBTC': 'WBTC',
    'XBT': 'WBTC',
}


profit1_omnipool = arb.calculate_profit(init_agent, agent1, asset_map_omnipool)
profit2_omnipool = arb.calculate_profit(init_agent, agent2, asset_map_omnipool)

print("strategy 1 profit")
print(profit1)
print("strategy 1 profit at Omnipool spot")
print(sum([profit1_omnipool[asset] * state1.price(state1, asset, 'USDT') for asset in profit1_omnipool]))
print("strategy 2 profit")
print(profit2)
print("strategy 2 profit at Omnipool spot")
print(sum([profit2_omnipool[asset] * state2.price(state2, asset, 'USDT') for asset in profit2_omnipool]))
            

strategy 1 profit
{'HDX': 0, 'USD': 26.546728491783142, 'BTC': 0.0002416372299194336, 'ETH': 0.0003020763397216797, 'DOT': 0.0, 'ASTR': 0, 'ZTG': 0, 'CFG': 0.0, 'BNC': 0.0, 'vDOT': 0, 'GLMR': 0, 'INTR': 0}
strategy 1 profit at Omnipool spot
34.66475301407448
strategy 2 profit
{'HDX': 0, 'USD': 3.292348027229309, 'BTC': 0.001155257225036621, 'ETH': 0, 'DOT': 0.0, 'ASTR': 0, 'ZTG': 0, 'CFG': 0.0, 'BNC': 0.0, 'vDOT': 0, 'GLMR': 0, 'INTR': 0}
strategy 2 profit at Omnipool spot
39.45737364780654
